# Exploring Reddit Data

The notebook demonstrates how to extract massive amounts of reddit data in order to build a map of subreddits. 

Data extracted from https://files.pushshift.io/reddit/comments/
- Files are around 2-10 times larger when uncompressed
- Uncompressed files must be renamed to have .json file extension

Code adapted from https://github.com/lmcinnes/subreddit_mapping

Credit to reddit user /u/Stuck_In_the_Matrix and /u/hoffa 

In [1]:
import json
import hdbscan
import sqlite3
import sqlalchemy
import adjustText
import subprocess
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.sparse as ss
from os.path import isfile
from sklearn.manifold import TSNE
from sklearn.utils import check_array
from sklearn.base import BaseEstimator
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD


import bokeh
from bokeh.plotting import figure, show, output_notebook, output_file
from bokeh.models import HoverTool, ColumnDataSource, value, CustomJS
from bokeh.models.mappers import LinearColorMapper
from bokeh.palettes import plasma
from collections import OrderedDict

from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from matplotlib.gridspec import GridSpec


# %matplotlib inline
sns.set_context('poster')
sns.set_style('white')
%config InlineBackend.figure_format = 'retina'

from sqlalchemy import create_engine
from IPython.display import clear_output

# Build SQL Database

Since the dataset is huge (71,826,552 comments), we need to read and write to the SQL database in chunks. 

In [2]:
def extract_comments(filepath, limit=None, links_only=False):
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (limit is not None) and (idx == limit):
                break
            comment = json.loads(line)
            if links_only:
                if 'https' in comment['body']:
                    comments.append(comment)
            else:
                comments.append(comment)
        return comments
    
    
def extract_subset(filepath, start=0, end=10):
    comments = []
    with open(filepath) as fp:
        for idx, line in enumerate(fp):
            if (idx >= start) and (idx < end):
                comment = json.loads(line)
                comments.append(comment)
            elif idx >= end:
                break                
        return comments
    
    
def write_to_database(db_conn, json_fp, chunk_size):
    batch_no=1
    for chunk in pd.read_json(json_fp, chunksize=chunk_size, lines=True):
        try: 
            chunk.to_sql('reddit_comments', db_conn, if_exists='append')
        except sqlalchemy.exc.SQLAlchemyError as e: 
            print("\n  {}".format(e.orig))
        print('\rindex: {}'.format(batch_no), end='')
        batch_no+=1
        
        
def create_database(database, n_files, comments_per_file, chunk_size):
    for idx in range(0, n_files):
        print("########## File {} ##########".format(idx+1))
        start = int(idx*comments_per_file)
        end = int(start + comments_per_file)
        print("Extracting comments {} - {}".format(start, end))
        comments = extract_subset(filepath='data/RC_2016-10.json', start=start, end=end)
        df = pd.DataFrame(comments)
        df.to_json("data/RC_2016-10_chunk.json", orient='records', lines=True)

        print("Writing to database")
        write_to_database(
            db_conn=database, 
            json_fp="data/RC_2016-10_chunk.json", 
            chunk_size=chunk_size
        )
        clear_output(wait=True)

In [3]:
conn = create_engine('sqlite:///reddit_database_small.db')

In [4]:
create_database(database=conn, n_files=20, comments_per_file=100000, chunk_size=1000)

########## File 20 ##########
Extracting comments 1900000 - 2000000
Writing to database
index: 100

### Run SQL queries

In [5]:
conn = sqlite3.connect('reddit_database_small.db')

#### TODO: check if SUM(1), COUNT(1) assumes zero indexing. Might be the root of the problem

Create a list of number of users in each subreddit, and save the result a new table

In [6]:
QUERY1 = """
CREATE TABLE subr_users AS
    SELECT subreddit, authors, DENSE_RANK() OVER (ORDER BY authors DESC) AS rank_authors
    FROM (SELECT subreddit, SUM(1) as authors
         FROM (SELECT subreddit, author, COUNT(1) as cnt 
             FROM reddit_comments
             WHERE author NOT LIKE '%bot'
             GROUP BY subreddit, author HAVING cnt > 0)
         GROUP BY subreddit) t
    ORDER BY authors DESC;
"""

c = conn.cursor()
c.execute(QUERY1)
conn.commit()

In [7]:
df = pd.read_sql_query("""SELECT * FROM subr_users""", conn)
df

,subreddit,authors,rank_authors
0,AskReddit,45394,1
1,leagueoflegends,12063,2
2,politics,11272,3
3,funny,9622,4
4,CFB,9325,5
...,...,...,...
18282,zinesters,1,619
18283,zocken,1,619
18284,zombiemanic,1,619
18285,zoophobiacomic,1,619


Using the new table, we create a list of number of users who authored at least 10 posts in pairs of subreddits

In [8]:
QUERY2 = """
CREATE TABLE overlapping_subr_users AS
    SELECT t1.subreddit, t2.subreddit, SUM(1) AS NumOverlaps
    FROM (SELECT subreddit, author, COUNT(1) AS cnt 
         FROM reddit_comments
         WHERE author NOT LIKE '%bot'
         AND subreddit IN (SELECT subreddit FROM subr_users
           WHERE rank_authors>200 AND rank_authors<2201)
         GROUP BY subreddit, author HAVING cnt > 10) t1
    JOIN (SELECT subreddit, author, COUNT(1) as cnt 
         FROM reddit_comments
         WHERE author NOT LIKE '%bot'
         GROUP BY subreddit, author HAVING cnt > 10) t2
    ON t1.author=t2.author
    WHERE t1.subreddit!=t2.subreddit
    GROUP BY t1.subreddit, t2.subreddit
"""

c = conn.cursor()
c.execute(QUERY2)
conn.commit()

Read in the result of the second query as a dataframe. Edit the column names and store it for later

In [9]:
df = pd.read_sql_query("""SELECT * FROM overlapping_subr_users""", conn)
df

,subreddit,subreddit:1,NumOverlaps
0,2b2t,2007scape,1
1,2b2t,2meirl4meirl,1
2,2b2t,3DS,1
3,2b2t,3Dprinting,1
4,2b2t,3dshacks,1
...,...,...,...
1498500,zootopia,yandere_simulator,1
1498501,zootopia,yokaiwatch,1
1498502,zootopia,youtubehaiku,1
1498503,zootopia,yugioh,1


In [10]:
df = df.rename(columns={"subreddit":"t1_subreddit", "subreddit:1":"t2_subreddit"})
df

,t1_subreddit,t2_subreddit,NumOverlaps
0,2b2t,2007scape,1
1,2b2t,2meirl4meirl,1
2,2b2t,3DS,1
3,2b2t,3Dprinting,1
4,2b2t,3dshacks,1
...,...,...,...
1498500,zootopia,yandere_simulator,1
1498501,zootopia,yokaiwatch,1
1498502,zootopia,youtubehaiku,1
1498503,zootopia,yugioh,1


In [11]:
df.to_json("data/RC_2016-10_subreddit_overlaps_small.json")

Close the database connection

In [12]:
conn.close()

# Subreddit Mapping

In [13]:
raw_data = pd.read_json("data/RC_2016-10_subreddit_overlaps_small.json")

In [14]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1498505 entries, 0 to 1498504
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   t1_subreddit  1498505 non-null  object
 1   t2_subreddit  1498505 non-null  object
 2   NumOverlaps   1498505 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 45.7+ MB


In [15]:
# raw_data.describe()

Try: rank the subreddits, take out the last few thousand

In [16]:
raw_data = raw_data.sort_values(by="NumOverlaps", ascending=False)

In [17]:
raw_data = raw_data.loc[raw_data.NumOverlaps > 2,:]

In [18]:
raw_data.head(60)

,t1_subreddit,t2_subreddit,NumOverlaps
562154,SVExchange,pokemontrades,15
116648,CFBOffTopic,CFB,12
132704,Cardinals,Dodgers,10
211441,Dodgers,Cardinals,10
509999,Pokemongiveaway,pokemontrades,8
1451977,vegasquadrantrp,ProjectFreelancerRP,8
83019,BankBallExchange,pokemontrades,8
519773,ProjectFreelancerRP,vegasquadrantrp,8
1210655,ockytop,CFB,7
134999,CasualPokemonTrades,pokemontrades,5


In [19]:
print("Number of pairwise commenter overlaps: {}".format(len(raw_data)))
print("t1_subreddit unique subreddits: {}".format(len(raw_data["t1_subreddit"].unique())))
print("t2_subreddit unique subreddits: {}".format(len(raw_data["t2_subreddit"].unique())))

Number of pairwise commenter overlaps: 65
t1_subreddit unique subreddits: 53
t2_subreddit unique subreddits: 40


Rank the subreddits so that they are indexed in order of popularity. Popularity is defined by the total number of unique commenters in each subreddit. 

In [20]:
# Use t1_subreddit if it is larger?
subreddit_popularity = raw_data.groupby('t2_subreddit')['NumOverlaps'].sum()
subreddits = np.array(subreddit_popularity.sort_values(ascending=False).index)

Pivot the data into a matrix such that rows and columns are both indexed by subreddits, and the entry at position (i,j) is the number of overlaps bwteen the ith and jth subreddits

Create subreddit-to-integer-index map to convert the subreddit names in the table into numeric row and column indexes.

In [21]:
index_map = dict(np.vstack([subreddits, np.arange(subreddits.shape[0])]).T)

In [22]:
for key, value in index_map.items():
    print(key, value)

CFB 0
pokemontrades 1
baseball 2
AskReddit 3
Dodgers 4
Pokemongiveaway 5
Cardinals 6
MLBStreams 7
ProjectFreelancerRP 8
CFBStreams 9
vegasquadrantrp 10
soccerstreams 11
CasualPokemonTrades 12
AFL 13
GWABackstage 14
The_Donald 15
SVExchange 16
gonewildaudio 17
GlobalOffensive 18
CringeAnarchy 19
AskMen 20
todayilearned 21
Battlefield 22
roleplayponies 23
politics 24
CampArcadia 25
CampHalfBloodRP 26
mylittlepony 27
movies 28
battlefield_4 29
hockey 30
bakchodi 31
IndiaSpeaks 32
NewYorkMets 33
PokemonPlaza 34
gonewild 35
buildapc 36
SuperMaM 37
TickTockManitowoc 38
worldnews 39


In [23]:
values = raw_data.NumOverlaps

In [24]:
row_indices = raw_data.t2_subreddit.map(index_map)

In [25]:
col_indices = raw_data.t1_subreddit.map(index_map)

In [26]:
row_indices

562154      1
116648      0
132704      4
211441      6
509999      1
           ..
1170080    23
1341031     9
761862     22
814834     30
482033     21
Name: t2_subreddit, Length: 65, dtype: int64

In [27]:
col_indices

562154     16.0
116648      NaN
132704      6.0
211441      4.0
509999      5.0
           ... 
1170080    27.0
1341031    11.0
761862     29.0
814834      NaN
482033      NaN
Name: t1_subreddit, Length: 65, dtype: float64

In [28]:
print(len(row_indices[row_indices.isna()==True]))
print(len(col_indices[col_indices.isna()==True]))

0
30


Create a sparse matrix. This format requires us to specify triples of row, column, and value for each non-zero entry in the matrix. The COO matrix constructor accepts this as a triple of arrays: the first array is the values, the second and third are arrays of row and column indices.

In [29]:
count_matrix = ss.coo_matrix((raw_data.NumOverlaps, 
                             (raw_data.t2_subreddit.map(index_map),
                              raw_data.t1_subreddit.map(index_map))),
                              shape=(subreddits.shape[0], subreddits.shape[0]),
                              dtype=np.float64)

ValueError: negative column index found

In [ ]:
conditional_prob_matrix = count_matrix.tocsr()
conditional_prob_matrix = normalize(conditional_prob_matrix, norm='l1', copy=False)

### Converting subreddit vectors into a map

In [ ]:
reduced_vectors = TruncatedSVD(n_components=500, random_state=1).fit_transform(conditional_prob_matrix)
reduced_vectors = normalize(reduced_vectors, norm='l2', copy=False)

In [ ]:
reduced_vectors.shape

In [ ]:
tsne = TSNE(verbose=1, n_components=2, perplexity=50)

subreddit_map = tsne.fit_transform(reduced_vectors[:10000])

In [ ]:
subreddit_map_df = pd.DataFrame(subreddit_map, columns=('x', 'y'))
subreddit_map_df['subreddit'] = subreddits[:10000]
subreddit_map_df.head()

### Clustering the map

In [ ]:
clusterer = hdbscan.HDBSCAN(min_samples=5, min_cluster_size=20).fit(subreddit_map)
cluster_ids = clusterer.labels_

In [ ]:
subreddit_map_df['cluster'] = cluster_ids

### Visualization

In [ ]:
output_notebook()
# To output to a static html file, comment out the previous
# line, and uncomment the line below.
# output_file('subreddit_interactive_map.html')

In [ ]:
# Construct a color palette and map clusters to colors
palette = ['#777777'] + list(plasma(cluster_ids.max()))

colormap = LinearColorMapper(palette=palette, low=-1, high=cluster_ids.max())

color_dict = {
    'field': 'cluster', 
    'transform': colormap
}

# Set fill alpha globally
subreddit_map_df['fill_alpha'] = np.exp((subreddit_map.min() - subreddit_map.max()) / 5.0) + 0.05

# Build a column data source
plot_data = ColumnDataSource(subreddit_map_df)

# Custom callback for alpha adjustment
jscode="""
    var data = source.data;
    var start = cb_obj.start;
    var end = cb_obj.end;
    alpha = data['fill_alpha']
    for (i = 0; i < alpha.length; i++) {
         alpha[i] = Math.exp((start - end) / 5.0) + 0.05;
    }
    source.trigger('change');
"""

# Create the figure and add tools
fig = figure(
    title='A Map of Subreddits',
    plot_width = 900,
    plot_height = 900,
    tools= ('pan, wheel_zoom, box_zoom,''box_select, reset'),
    active_scroll=u'wheel_zoom'
)

# tools= ('pan, wheel_zoom, box_zoom,''box_select, resize, reset'),
fig.add_tools(
    HoverTool(
        tooltips = OrderedDict(
            [('subreddit', '@subreddit'), ('cluster', '@cluster')]
        )
    )
)

# draw the subreddits as circles on the plot
fig.circle(
    u'x', u'y', 
    source = plot_data,
    fill_color = color_dict, 
    line_color = None, 
    fill_alpha = 'fill_alpha',
    size = 10, 
    hover_line_color = u'black'
)

callback = CustomJS(
    args=dict(
        source=plot_data,
    ), 
    code=jscode
)

# fig.x_range.callback = CustomJS(args=dict(source=plot_data), code=jscode)
# fig.y_range.callback = CustomJS(args=dict(source=plot_data), code=jscode)

# fig.x_range.js_on_change("plot_data", callback)
# fig.y_range.js_on_change("plot_data", callback)

# configure visual elements of the plot
fig.title.text_font_size = value('18pt')
fig.title.align = 'center'
fig.xaxis.visible = False
fig.yaxis.visible = False
fig.grid.grid_line_color = None
fig.outline_line_color = '#222222'

# # display the figure
show(fig)

### Exploring clusters

In [ ]:
def cluster_bounds(dataframe, subreddit):
    # Find the cluster the subreddit belongs to
    cluster = dataframe.cluster[
        dataframe.subreddit == subreddit].values[0]
    if cluster == -1:
        warn('This subreddit was lost as noise and not in any cluster')
        
    # Extract the dubset of the dataframe that is the cluster
    sub_dataframe = dataframe[dataframe.cluster == cluster]
    
    x_min = sub_dataframe.x.min()
    x_max = sub_dataframe.x.max()
    x_padding = (x_max - x_min) * 0.5
    x_min -= x_padding
    x_max += x_padding
    
    y_min = sub_dataframe.y.min()
    y_max = sub_dataframe.y.max()
    y_padding = (y_max - y_min) * 0.5
    y_min -= y_padding
    y_max += y_padding

    return x_min, x_max, y_min, y_max


def data_in_bounds(dataframe, bounds):
    return dataframe[
        (dataframe.x > bounds[0]) &
        (dataframe.x < bounds[1]) &
        (dataframe.y > bounds[2]) &
        (dataframe.y < bounds[3])
    ]


def plot_cluster(dataframe, subreddit):
    # Build a color map to match the Bokeh plot
    colormap = dict(zip(
        np.unique(dataframe.cluster),
        ['#777777'] + sns.color_palette('plasma', dataframe.cluster.max() + 1).as_hex()
    ))
    subregion_defined = True
    
    # Figure and gridspec to layout axes
    fig = plt.figure(figsize=(16,10))
    gs = GridSpec(3, 3)
    
    # First axes, spanning most of the figure
    # Contains just the points in a region 
    # around the points in the cluster
    ax1 = plt.subplot(gs[:,:2])
    try:
        bounds = cluster_bounds(dataframe, subreddit)
    except IndexError:
        ax1.text(0.5, 0.5, 'Subreddit {} not found!'.format(subreddit), 
                 horizontalalignment='center', verticalalignment='center',
                 transform=ax1.transAxes, fontsize=18)
        subregion_defined = False
    
    if subregion_defined:
        to_plot = data_in_bounds(dataframe, bounds)
        ax1.scatter(to_plot.x, to_plot.y, c=to_plot.cluster.map(colormap), s=30, alpha=0.5)
    
        # We want to add text labels. We subsample up to 50 labels
        # And then use adjustText to get them non-overlapping
        text_elements = []
        for row in to_plot.sample(n=min(len(to_plot),50), random_state=0).values:
            if row[2] != subreddit:
                text_elements.append(ax1.text(row[0], row[1], row[2], alpha=0.5, fontsize=9))
        row = to_plot[to_plot.subreddit == subreddit].values[0]
        text_elements.append(ax1.text(row[0], row[1], row[2], 
                                      color='g',
                                      alpha=0.5, fontsize=11))
        adjustText.adjust_text(text_elements, ax=ax1, lim=100,
                               force_text=0.1, force_points=0.1,
                               arrowprops=dict(arrowstyle="-", color='k', lw=0.5))
    
    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])

    # Second axes, center right of the figure
    # Plots all the data and a rectangle
    # Showing the area selected out
    ax2 = plt.subplot(gs[1,2])
    ax2.scatter(dataframe.x, dataframe.y, s=20,
                c=dataframe.cluster.map(colormap), alpha=0.05)
    
    if subregion_defined:
        ax2.add_patch(Rectangle(xy=(bounds[0], bounds[2]),
                                    width=(bounds[1] - bounds[0]),
                                    height=(bounds[3] - bounds[2]),
                                    edgecolor='k', facecolor='none', lw=1))
    ax2.xaxis.set_ticklabels([])
    ax2.yaxis.set_ticklabels([])
    plt.tight_layout()

    if subregion_defined:
        # Now we make use of the power of matplotlib transforms
        # to draw line from the subselected rectangle in axes2
        # all the way to the bounds of axes1
        trans_figure = fig.transFigure.inverted()

        ax1_coord = trans_figure.transform(ax1.transAxes.transform((1,0)))
        ax2_coord = trans_figure.transform(ax2.transData.transform((bounds[1],bounds[2])))
        connector1 = Line2D((ax1_coord[0],ax2_coord[0]),(ax1_coord[1],ax2_coord[1]),
                              transform=fig.transFigure, lw=1, color='k')
        ax1_coord = trans_figure.transform(ax1.transAxes.transform((1,1)))
        ax2_coord = trans_figure.transform(ax2.transData.transform((bounds[1],bounds[3])))
        connector2 = Line2D((ax1_coord[0],ax2_coord[0]),(ax1_coord[1],ax2_coord[1]),
                              transform=fig.transFigure, lw=1, color='k')

        fig.lines = [connector1, connector2]

In [ ]:
plot_cluster(subreddit_map_df, 'trump')

In [ ]:
plot_cluster(subreddit_map_df, 'MachineLearning')

In [ ]:
from ipywidgets import interact_manual, fixed, Text

In [ ]:
interact_manual(plot_cluster, 
                dataframe=fixed(subreddit_map_df), 
                subreddit=Text())

In [ ]:
coherent_clusters = np.argsort(clusterer.cluster_persistence_)[-10:][::-1]
coherence = np.sort(clusterer.cluster_persistence_)[-10:][::-1]
fig = plt.figure(figsize=(12,5))
ax = plt.gca()
ax.bar(np.arange(10), coherence)
ax.set_xticks(np.arange(10))
ax.set_xticklabels(coherent_clusters);

In [ ]:
def plot_cluster_by_id(dataframe, cluster_id):
    subreddits_in_cluster = np.array(dataframe.subreddit[cluster_ids == cluster_id])
    plot_cluster(dataframe, subreddits_in_cluster[0])
    plt.gcf().text(0.5, 0.98, 'Cluster {}'.format(cluster_id), ha='center')

In [ ]:
plot_cluster_by_id(subreddit_map_df, 2)